# Modelling v3

modelling use the latest exploration

# Goal

Get the model from the prepared data. With undersampling + sample weight. 

# Plan

Checklist what will be done on the notebook :

    [*] Get Data
    [*] Modelling



In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
import time

from scipy.stats import uniform,randint
import warnings
warnings.filterwarnings("ignore")

In [2]:
######### run manually #############

# with open("../../config.yaml", "r") as f:
#     config = yaml.load(f, Loader=yaml.FullLoader)

# PATH_RAW = config['path']['raw']
# PATH_INTERIM = config['path']['interim']
# PATH_PROCESSED = config['path']['processed']

# if os.name == 'posix' :
#     PATH_MODEL = config['path']['artifact']+os.getcwd().split('/')[-1]+'/'
# else:
#     PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'

# PATH_UDF = config['path']['udf']

######### run with makefile #############

with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

PATH_RAW = config['path-make']['raw']
PATH_INTERIM = config['path-make']['interim']
PATH_PROCESSED = config['path-make']['processed']

if os.name == 'posix' :
    PATH_MODEL = config['path-make']['artifact']+'iteration_2/'
else:
    PATH_MODEL = config['path-make']['artifact']+'iteration_2/'

PATH_UDF = config['path-make']['udf']

In [3]:
from sklearn.linear_model import LogisticRegression, ElasticNet, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessRegressor

## Get Data

In [4]:
raw_data = pd.read_csv(PATH_PROCESSED+'prep_iter2.csv')
col_cat_load = joblib.load(PATH_MODEL+'cat_names.pkl')
col_num_load = joblib.load(PATH_MODEL+'num_names.pkl')

In [5]:
all_tr = raw_data[raw_data.is_train == 1]
down_size = int(len(all_tr[all_tr.target == 1]) * 1.5)
all_tr0 = all_tr[all_tr.target == 0].sample(down_size,random_state = 125)

all_tr0['sweight'] = 0.4/0.116
all_tr1 = all_tr[all_tr.target == 1]
all_tr1['sweight'] = 1

all_tr_down = pd.concat([all_tr0,all_tr1])
all_tr_down.target.value_counts(normalize = 1)

samp_w = all_tr_down['sweight']

In [6]:
X_tr = all_tr_down[col_num_load+col_cat_load]
Y_tr = all_tr_down['target'].values

X_val = raw_data[raw_data.is_train == 0][col_num_load+col_cat_load]
Y_val = raw_data[raw_data.is_train == 0]['target'].values

In [7]:
X_tr.shape

(9430, 26)

In [8]:
X_val.shape

(5765, 26)

In [9]:
Y_val.shape

(5765,)

## Modelling

In [10]:
proportion_cls = pd.Series(Y_tr).value_counts(normalize = True)
scale_pos_w = np.ceil(proportion_cls[0]/proportion_cls[1])
cw = {0:1, 1:scale_pos_w}
cw
# scale_pos_weight for xgboost

{0: 1, 1: 2.0}

In [11]:
proportion_cls

0    0.6
1    0.4
dtype: float64

In [12]:
all_model = {'sgd3' : SGDClassifier(class_weight=cw,max_iter = 2500,early_stopping=True,warm_start=True,eta0=0.1)
            ,'logreg3' : LogisticRegression(class_weight=cw)
            ,'rf3': RandomForestClassifier(class_weight=cw, random_state=234)
            ,'nb3':GaussianNB(priors=[0.884,0.116])
            }

In [13]:
param_options = {'logreg3' : dict(C=uniform(loc=0, scale=2)
                                 ,penalty=['l2', 'l1']
                                 ,solver=['liblinear', 'sag'])
                 ,'rf3' : dict(n_estimators=randint(300, 500)
                             ,max_depth=randint(20, 50)
                             ,min_samples_split=randint(5, 20)
                             ,min_samples_leaf=randint(5, 20)
                             ,min_weight_fraction_leaf=uniform(loc=0, scale=0.4)
                             ,min_impurity_decrease=uniform(loc=0, scale=0.3))
                 ,'nb3' : dict(var_smoothing = uniform(loc=1e-8, scale=1e-2))
                 # ,'gp2' : dict(alpha = uniform(loc=1e-12, scale=1e-5))
                 ,'sgd3' : dict(loss=['hinge', 'squared_hinge', 'perceptron']
                                ,penalty=['l2', 'l1']
                                ,alpha=uniform(loc=0.1, scale=0.95)
                                ,l1_ratio=uniform(loc=0.1, scale=0.75)
                                ,learning_rate=['constant','optimal','invscaling','adaptive']
                               )
                 # ,'el2': dict(alpha=uniform(loc=0.1, scale=2), l1_ratio=uniform(loc=0.1, scale=2))
                }

 # min_weight_fraction_leaf must in [0, 0.5]

In [14]:
## scoring
## https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
best_clf = {}
best_scr = {}
sla_model = {}
for mdl in all_model.keys():
    L1 = time.time()
    clf = all_model[mdl]
    parm = param_options[mdl]
    RS = RandomizedSearchCV(clf, parm, n_iter = 300, cv = 5, scoring = 'roc_auc',n_jobs = 10)
        
    RS.fit(X_tr,Y_tr, sample_weight=samp_w)
    
    best_clf[mdl] = RS.best_estimator_
    best_scr[mdl] = RS.best_score_
    diff_time = round((time.time()-L1)/60, 2)
    
    joblib.dump(best_clf[mdl],PATH_MODEL+mdl+'.pkl')
    
    print(mdl, 'is done in ',diff_time, ' minutes')
    sla_model[mdl] = diff_time

sgd3 is done in  0.29  minutes


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalt

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estim

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estim

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalt

...

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for t

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 444, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for t

logreg3 is done in  0.66  minutes


rf3 is done in  3.68  minutes


nb3 is done in  0.09  minutes


In [15]:
sla_model

{'sgd3': 0.29, 'logreg3': 0.66, 'rf3': 3.68, 'nb3': 0.09}

In [16]:
best_scr

{'sgd3': 0.8337135557838286,
 'logreg3': 0.8363296463846834,
 'rf3': 0.8047440615420036,
 'nb3': 0.7599924783301194}

In [17]:
best_clf

{'sgd3': SGDClassifier(alpha=0.1558429822201508, class_weight={0: 1, 1: 2.0},
               early_stopping=True, eta0=0.1, l1_ratio=0.15471548212988842,
               learning_rate='invscaling', loss='squared_hinge', max_iter=2500,
               warm_start=True),
 'logreg3': LogisticRegression(C=1.9527291773428896, class_weight={0: 1, 1: 2.0},
                    solver='sag'),
 'rf3': RandomForestClassifier(class_weight={0: 1, 1: 2.0}, max_depth=25,
                        min_impurity_decrease=0.0025377145137406407,
                        min_samples_leaf=18, min_samples_split=9,
                        min_weight_fraction_leaf=0.14313554756048644,
                        n_estimators=301, random_state=234),
 'nb3': GaussianNB(priors=[0.884, 0.116], var_smoothing=0.009154214146774826)}

## Evaluation

The SVM is best so far, but there is no predict proba.

In [18]:
for ml in best_clf.keys():
    Y_prob_pred = best_clf[ml].predict(X_val)
    print(ml)
    print(classification_report(Y_val,Y_prob_pred))

sgd3
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      5079
           1       0.47      0.46      0.47       686

    accuracy                           0.87      5765
   macro avg       0.70      0.70      0.70      5765
weighted avg       0.87      0.87      0.87      5765

logreg3


              precision    recall  f1-score   support

           0       0.94      0.91      0.93      5079
           1       0.47      0.57      0.51       686

    accuracy                           0.87      5765
   macro avg       0.70      0.74      0.72      5765
weighted avg       0.88      0.87      0.88      5765

rf3
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      5079
           1       0.00      0.00      0.00       686

    accuracy                           0.88      5765
   macro avg       0.44      0.50      0.47      5765
weighted avg       0.78      0.88      0.83      5765

nb3
              precision    recall  f1-score   support

           0       0.94      0.81      0.87      5079
           1       0.29      0.59      0.39       686

    accuracy                           0.78      5765
   macro avg       0.62      0.70      0.63      5765
weighted avg       0.86      0.78      0.81      5765



In [19]:
1-Y_val.mean()

0.8810060711188205